In [33]:
from qoop.compilation.qsp import QuantumStatePreparation
import numpy as np
import qiskit
from qiskit.primitives import Sampler
from qiskit.circuit.library import StatePreparation

def q_inner_product(state1, state2):
    num_qubits = int(np.log2(len(state1)))
    qc = qiskit.QuantumCircuit(1 + num_qubits * 2, 1)
    qc.h(0)
    # Replace by QuantumCompilation
    qc.append(StatePreparation(state1), range(1, num_qubits + 1))
    qc.append(StatePreparation(state2), range(num_qubits + 1, num_qubits * 2 + 1))
    for i in range(num_qubits):
        qc.cswap(0, i + 1, i + num_qubits + 1)
    qc.h(0)
    qc.measure(0, 0)
    sampler = Sampler()
    job = sampler.run(qc)
    counts = job.result().quasi_dists[0]
    return np.sqrt(counts.get(0,0) - counts.get(1, 0))

def compute_Lvi(L, norm_L, v, norm_v):
    return np.squeeze(
        np.array(
            [norm_v * norm_L[i] * q_inner_product(L[i, :], v) for i in range(L.shape[0])]
            )
        )
    
def compute_Qmatmul(origin_L, origin_v):
	norm_L =  np.linalg.norm(origin_L, axis=1, keepdims=True)
	norm_v = np.linalg.norm(origin_v)
	L = origin_L / norm_L
	v = origin_v / norm_v
	Lvi = compute_Lvi(L, norm_L, v, norm_v)
	return Lvi

size = 4
origin_L = np.random.uniform(1, 4, (size, size))
origin_v = np.random.uniform(1, 2, size)

print('Quantum matmul result:', compute_Qmatmul(origin_L, origin_v))
print('Origin matmul result: ', origin_L @ origin_v)
    

Quantum matmul result: [14.84210791 13.56305474 15.81207483 20.1856222 ]
Origin matmul result:  [14.84210791 13.56305474 15.81207483 20.1856222 ]


/var/folders/bw/2cm4rnhn0djgg95n4py1pysc0000gp/T/ipykernel_2154/390912436.py:18: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()


In [20]:
import matplotlib.pyplot as plt
from qoop.pcompilation.pcompilation import pcompilation
from qoop.core.ansatz import WchainCNOT_xyz
def prepare_state(v):
    num_qubits = int(np.log2(len(v)))
    qsp = QuantumStatePreparation(
        u = WchainCNOT_xyz(num_qubits, num_qubits), 
        target_state = v).fit()
    return qsp.compiler.thetass[-1]

In [21]:
size = 4
origin_L = np.random.uniform(1, 4, (size, size))
origin_v = np.random.uniform(1, 2, size)
norm_L =  np.linalg.norm(origin_L, axis=1, keepdims=True)
norm_v = np.linalg.norm(origin_v)
L = origin_L / norm_L
v = origin_v / norm_v
num_qubits = int(np.log2(len(origin_v)))
US = [0]*size
V = prepare_state(v)
for i in range(size):
    US[i] = prepare_state(L[i, :])


/Users/vutuanhai/Documents/GitHub/QuantumNCut/qoop/core/state.py:273: UserWarning: The input state is not normalized, we will normalize it for you
  warnings.warn("The input state is not normalized, we will normalize it for you")


In [23]:
for i in range(size):
    sub_qc = WchainCNOT_xyz(num_qubits, num_qubits)
    qc = qiskit.QuantumCircuit(1 + 2 * num_qubits, 1)
    qc.h(0)
    qc.append(sub_qc.assign_parameters(US[i]), range(1, num_qubits + 1))
    qc.append(sub_qc.assign_parameters(V), range(num_qubits + 1, 2*num_qubits + 1))
    for j in range(num_qubits):
        qc.cswap(0, j + 1, j + num_qubits + 1)
    qc.h(0)
    qc.measure(0, 0)
    sampler = Sampler()
    job = sampler.run(qc)
    counts = job.result().quasi_dists[0]
    print(norm_v * norm_L[i] * np.sqrt(counts.get(0,0) - counts.get(1, 0)))

[14.28819358]
[10.32859382]
[17.50969]
[14.29037255]


/var/folders/bw/2cm4rnhn0djgg95n4py1pysc0000gp/T/ipykernel_2154/1866361214.py:11: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()


In [24]:
print(origin_L @ origin_v)

[14.29574851 10.33873504 17.50450472 14.29080338]
